In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("../data/data_imputed_2.csv")

In [3]:
df.head()

,timestamp_local,temp,city_name,country_code,aqi,co,no2,o3,pm10,pm25,so2
0,01/02/2022 0:00,12.6,Gujrāt,PK,385,1339.8,76.0,10.7,491.7,347.67,238.0
1,01/02/2022 1:00,11.5,Gujrāt,PK,404,1437.6,76.0,9.3,508.3,359.33,268.0
2,01/02/2022 2:00,11.9,Gujrāt,PK,421,1535.5,76.0,8.0,525.0,371.00,298.0
3,01/02/2022 3:00,12.2,Gujrāt,PK,425,1659.0,68.3,5.3,529.3,374.00,275.7
4,01/02/2022 4:00,11.9,Gujrāt,PK,430,1782.5,60.7,2.7,533.7,377.00,253.3


In [4]:
from sklearn.model_selection import train_test_split

In [5]:
data = df[['pm10']]

In [6]:
from sklearn.preprocessing import MinMaxScaler
values = data.values

values = values.astype('float32')

scaler = MinMaxScaler(feature_range=(0, 1))
scaled = scaler.fit_transform(values)

In [7]:
scaled = scaled.reshape((19706,))

In [8]:
scaled.shape

(19706,)

In [9]:
def df_to_X_y(df, window_size, forecast_length):
    # df_as_np = df.to_numpy()
    X = []
    y = []
    
    for i in range(len(df) - window_size - forecast_length):
        row_x = [[a] for a in df[i:i+window_size]]
        
        X.append(row_x)
        # row_y = df_as_np[i+window_size]
        row_y = [b for b in df[i+window_size:i+window_size+forecast_length]]
        
        y.append(row_y)
        
    return np.array(X), np.array(y)

In [10]:
WINDOW_WIDTH = 168
FORECAST_LENGTH = 168
X, y = df_to_X_y(scaled, WINDOW_WIDTH, FORECAST_LENGTH)


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

X_max = X_train.max()
X_min = X_train.min()

In [11]:
batchSize = 16
from keras.models import Sequential, save_model, load_model
from keras.layers import *
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from keras.losses import MeanSquaredError
from keras.metrics import RootMeanSquaredError

In [12]:
normalizer = Normalization()
normalizer.adapt(X_train)
rlrop = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10)

model = Sequential()
# 1D Convolutional layers
model.add(Conv1D(filters=8, kernel_size=3, activation='relu', input_shape=X_train.shape[1:]))
# model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
# model.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
# Flatten the output of the Conv layers
model.add(Flatten())
# model.add(LSTM(input_shape = X_train.shape[1:], units=86, return_sequences=True))
# model.add(LSTM(units=64, return_sequences=True))
# model.add(GRU(units=86, return_sequences=True))
# model.add(LSTM(units=128, return_sequences=True))
# model.add(GRU(units=100, return_sequences=True))
# model.add(LSTM(units=100, return_sequences=False))
# model.add(Dense(units=100,  activation='relu'))
# model.add(Dense(8, activation='relu'))

model.add(Dense(units=FORECAST_LENGTH, activation='linear'))

model.summary()

model.compile(optimizer=Adam(learning_rate=0.001), metrics=[RootMeanSquaredError()] , loss=MeanSquaredError())

history = model.fit(X_train, y_train, validation_data=(X_val,y_val), epochs=50, callbacks=[rlrop])

# score = model.evaluate(X_test, y_test, verbose = 1)

C:\Users\Hassan\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                      │ (None, 166, 8)              │              32 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 1328)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 168)                 │         223,272 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 223,304 (872.28 KB)

 Trainable params: 223,304 (872.28 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
388/388 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - loss: 0.0076 - root_mean_squared_error: 0.0859 - val_loss: 0.0050 - val_root_mean_squared_error: 0.0708 - learning_rate: 0.0010
Epoch 2/50
388/388 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.0050 - root_mean_squared_error: 0.0709 - val_loss: 0.0049 - val_root_mean_squared_error: 0.0703 - learning_rate: 0.0010
Epoch 3/50
388/388 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.0050 - root_mean_squared_error: 0.0706 - val_loss: 0.0050 - val_root_mean_squared_error: 0.0707 - learning_rate: 0.0010
Epoch 4/50
388/388 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 0.0049 - root_mean_squared_error: 0.0700 - val_loss: 0.0049 - val_root_mean_squared_error: 0.0697 - learning_rate: 0.0010
Epoch 5/50
388/388 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.0046 - root_mean_squared_error: 0.0680 - val_loss: 0.0050 - val_root_mean_squared_error: 0.0704 - learning_rate: 0.0010
Epoch 6/50
388/388 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - loss: 0.0048 - root_mean_squared_err

In [13]:
forecast = model.predict(X_test)

122/122 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


In [14]:
poll = np.array(df["pm10"])

meanop = poll.mean()
stdop = poll.std()

y_test_true = y_test*stdop + meanop
testPredict = forecast*stdop + meanop

In [15]:
from sklearn.metrics import mean_squared_error

In [16]:
rmse = np.sqrt(mean_squared_error(y_test_true, testPredict))
print("Test RMSE ="  ,rmse)

Test RMSE = 5.764044


In [17]:
# Calculate MAPE
mape = np.mean(np.abs((y_test_true - testPredict) / y_test_true)) * 100
print("Test MAPE =", mape)

Test MAPE = 3.0447712168097496


In [18]:
# Calculate accuracy as 1 - MAPE
accuracy = 100 - mape
print("Accuracy =", accuracy, "%")

Accuracy = 96.95522878319025 %


In [27]:
from sklearn.metrics import mean_absolute_error

# Calculate MAE
mae = mean_absolute_error(y_test_true, testPredict)
print("Test MAE =", mae)

Test MAE = 4.046876


In [19]:
y_test_true.shape

(3874, 168)

In [20]:
testPredict.shape

(3874, 168)

In [21]:
import numpy as np

def calculate_segment_rmse(predictions, actuals, segment_length=1):
    num_segments = predictions.shape[1] // segment_length
    rmses = []
    
    for i in range(num_segments):
        start_idx = i * segment_length
        end_idx = start_idx + segment_length
        
        # Extract the segment for both predictions and actuals
        pred_segment = predictions[:, start_idx:end_idx]
        actual_segment = actuals[:, start_idx:end_idx]
        
        # Calculate RMSE for this segment
        segment_rmse = np.sqrt(mean_squared_error(actual_segment.flatten(), pred_segment.flatten()))
        rmses.append(segment_rmse)
    
    return rmses

In [22]:
segment_rmses = calculate_segment_rmse(testPredict, y_test_true)
for i, rmse in enumerate(segment_rmses):
    print(f"RMSE for segment {i + 1}: {rmse:.4f}")

RMSE for segment 1: 1.2969
RMSE for segment 2: 2.0122
RMSE for segment 3: 2.7062
RMSE for segment 4: 3.2232
RMSE for segment 5: 3.6736
RMSE for segment 6: 4.0149
RMSE for segment 7: 4.2646
RMSE for segment 8: 4.4217
RMSE for segment 9: 4.5521
RMSE for segment 10: 4.6421
RMSE for segment 11: 4.6828
RMSE for segment 12: 4.7208
RMSE for segment 13: 4.7923
RMSE for segment 14: 4.8546
RMSE for segment 15: 4.9311
RMSE for segment 16: 4.9488
RMSE for segment 17: 4.9185
RMSE for segment 18: 4.9097
RMSE for segment 19: 4.8714
RMSE for segment 20: 4.8459
RMSE for segment 21: 4.8425
RMSE for segment 22: 4.8490
RMSE for segment 23: 4.8585
RMSE for segment 24: 4.9048
RMSE for segment 25: 4.9986
RMSE for segment 26: 5.0401
RMSE for segment 27: 5.0999
RMSE for segment 28: 5.1737
RMSE for segment 29: 5.2602
RMSE for segment 30: 5.3290
RMSE for segment 31: 5.3655
RMSE for segment 32: 5.3886
RMSE for segment 33: 5.4540
RMSE for segment 34: 5.4918
RMSE for segment 35: 5.4510
RMSE for segment 36: 5.4235
R

In [23]:
forecast

array([[0.20608689, 0.19107956, 0.17919657, ..., 0.16581467, 0.1559532 ,
        0.15212072],
       [0.12087886, 0.12230468, 0.12287796, ..., 0.19474727, 0.1842819 ,
        0.18164772],
       [0.07243351, 0.07161169, 0.06903677, ..., 0.08043282, 0.07799489,
        0.07745362],
       ...,
       [0.18405525, 0.18701963, 0.19909997, ..., 0.20925198, 0.1930792 ,
        0.17919481],
       [0.5287624 , 0.50689787, 0.42252597, ..., 0.45795754, 0.45399064,
        0.43755305],
       [0.05769374, 0.0603647 , 0.06228498, ..., 0.07378663, 0.07404203,
        0.07492283]], dtype=float32)

In [24]:
testPredict

array([[133.62717 , 132.31496 , 131.27594 , ..., 130.10587 , 129.2436  ,
        128.90851 ],
       [126.176796, 126.30147 , 126.35159 , ..., 132.63565 , 131.72058 ,
        131.49026 ],
       [121.940865, 121.86901 , 121.64387 , ..., 122.640305, 122.42714 ,
        122.379814],
       ...,
       [131.70078 , 131.95996 , 133.01624 , ..., 133.9039  , 132.4898  ,
        131.27579 ],
       [161.84103 , 159.92926 , 152.552   , ..., 155.65005 , 155.30319 ,
        153.86594 ],
       [120.65206 , 120.885605, 121.053505, ..., 122.05918 , 122.08151 ,
        122.15852 ]], dtype=float32)

In [25]:
y_test_true

array([[132.39545 , 132.6498  , 132.90416 , ..., 126.41791 , 125.654816,
        124.89172 ],
       [126.290726, 126.290726, 126.290726, ..., 131.88672 , 130.32237 ,
        128.74533 ],
       [121.674034, 120.98725 , 120.313194, ..., 123.238365, 123.36555 ,
        123.49273 ],
       ...,
       [131.12363 , 132.14108 , 133.15852 , ..., 149.48863 , 144.47769 ,
        139.47945 ],
       [159.52325 , 156.17839 , 152.61731 , ..., 151.43451 , 149.94649 ,
        149.78116 ],
       [120.86007 , 121.41967 , 121.96655 , ..., 120.69474 , 121.71219 ,
        122.729645]], dtype=float32)

In [24]:
model.save("pm10_1dcnn.keras")